# Meeting Note Summarizer

Before starting Jupyter, we need to set the API keys to `init_env_vars.sh` file and source it as `source init_env_vars.sh` to make the environment variables available.

## The technology stack

### Speaker diarization
- AssemblyAI

### Speech-to-text
- Replicate, Whisper

### next?

In [4]:
import os
import gradio as gr

# Change working directory
if os.getcwd().split('/')[-1] == 'notebooks':
    print(f"Currently in notebooks folder. Switching to root.")
    os.chdir('../')

## AssemblyAI

## Gradio demo

Define transcription function for AssemblyAI

In [9]:
from pathlib import Path

filepath = './data/test.wav'
model = replicate.models.get("openai/whisper")
version = model.versions.get("23241e5731b44fcb5de68da8ebddae1ad97c5094d24f94ccb11f7c1d33d661e2")
output = version.predict(audio=Path(filepath))

In [5]:
from src.api import transcribe

with gr.Blocks() as demo:
    gr.HTML("""
    <center>
        <h1>NotePal</h1>
        <p>This is application</p>
    </center>
    """)
    
    with gr.Tab("Provide audio or video file"):
        mic = gr.Audio(source="microphone", type="filepath")
        upload_audio = gr.Audio(source="upload", type="filepath")
        upload_video = gr.Video()
        submit = gr.Button("Submit", variant='primary')
        
    with gr.Tab("Raw transcription"):
        output = gr.Textbox(label="Raw output")
        
    with gr.Tab("Analyze text"):
        aai_summary = gr.Textbox(label="AAI summary")
        openai_summary = gr.Textbox(label="OpenAI summary")
    
    with gr.Tab("Timeline"):
        chapters = gr.Textbox(label="Chapters")
    
    with gr.Tab('Speaker diarization'):
        diarization = gr.Textbox(label="Diarization")
    
    submit.click(transcribe, inputs=[mic, upload_audio, upload_video], 
                 outputs=[output, aai_summary, openai_summary, diarization, chapters])
    
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


File uploaded in 0.0017673804759979247s
